In [1]:
import awswrangler as wr
import psycopg2
import pandas as pd
from tqdm import tqdm
import multiprocessing
import os
from psycopg2 import pool
import concurrent.futures

credentials = wr.secretsmanager.get_secret_json("jupyterhub/terentev-a/credentials_serverless.json")

In [2]:
projects = ('gs_as', 'gs_gp', 'hs_as', 'hs_gp', 'mm_as', 'mm_gp', 'fd_as', 'fd_gp')
project_ids = {
  'as':
       {'gs': {'imp_id': 3789, 'app_short': 'gs_as'},
       'fd': {'imp_id': 3444, 'app_short': 'fd_as'},
       'ts': {'imp_id': 758, 'app_short': 'ts_as'},
       'hs': {'imp_id': 4698, 'app_short': 'hs_as'},
       'ws': {'imp_id': 3928, 'app_short': 'ws_as'},
       'mm': {'imp_id': 30975, 'app_short': 'mm_as'},
       'fds': {'imp_id': 31829, 'app_short': 'fds_as'}},
  'gp':
       {'gs': {'imp_id': 3790, 'app_short': 'gs_gp'},
       'fd': {'imp_id': 3426, 'app_short': 'fd_gp'},
       'ts': {'imp_id': 1170, 'app_short': 'ts_gp',},
       'hs': {'imp_id': 4699, 'app_short': 'hs_gp'},
       'ws': {'imp_id': 4761, 'app_short': 'ws_gp'},
       'mm': {'imp_id': 31756, 'app_short': 'mm_gp'},
       'fds': {'imp_id': 31897, 'app_short': 'fds_gp'}},
  'az':
      {'gs': {'imp_id': 3791, 'app_short': 'gs_az'},
       'fd': {'imp_id': 3606, 'app_short': 'fd_az_2'},
       'ts': {'imp_id': 1188, 'app_short': 'ts_az'},
       'hs': {'imp_id': 4700, 'app_short': 'hs_az'},
       'ws': {'imp_id': 31119, 'app_short': 'ws_az'},
       'mm': {'imp_id': 31761, 'app_short': 'mm_az'}}
}

#### Create table

In [12]:
con = psycopg2.connect(dbname='dev', 
                       host=credentials['host'], 
                       port=credentials['port'], 
                       user=credentials['username'], 
                       password=credentials['password']
                      )
cur = con.cursor()
cur.execute(f"""
drop table if exists ma_data.terentev_reinstalls_last_active;
create table ma_data.terentev_reinstalls_last_active (
    app_short varchar(256),
    user_id varchar(256),
    install_date date,
    install_time timestamp,
    country_code varchar(2),
    media_source varchar(256),
    last_active timestamp
)

DISTSTYLE KEY
DISTKEY (user_id)
""")
cur.close()
con.commit()
con.close()

In [3]:
def insert_table(s):
    d = s[1]
    p = s[0]
    name, platform = p.split('_')
    project_id =  project_ids[platform][name]['imp_id']
    m = d.strftime('%Y-%m-01')
    d = d.strftime('%Y-%m-%d')        

    con = psycopg2.connect(dbname='dev', 
                           host=credentials['host'], 
                           port=credentials['port'], 
                           user=credentials['username'], 
                           password=credentials['password']
                          )
    cur = con.cursor()
    cur.execute(f"""
    insert into ma_data.terentev_reinstalls_last_active

    with reinstalls as (
        select
            customer_user_id,
            country_code,
            install_time::DATETIME,
            media_source,
            'paid' as type
        from
            plr.af_raw_data.reinstalls u
        where
            month = '{m}'
            and date(install_time) = '{d}'
            and app_short = '{p}'
        union all
        select
            customer_user_id,
            country_code,
            install_time::DATETIME,
            media_source,
            'organic' as type
        from
            plr.af_raw_data.organic_reinstalls u
        where
            month = '{m}'
            and date(install_time) = '{d}'
            and app_short = '{p}'
        union all
        select
            customer_user_id,
            country_code,
            event_time::DATETIME as install_time,
            media_source,
            'paid' as type
        from
            plr.af_raw_data.conversions_retargeting
        where
            month = '{m}'
            and date(event_time) = '{d}'
            and app_short = '{p}'
            and retargeting_conversion_type = 'reattribution'
            and event_name = 'install'),

    reinstalls_unique as (
        select
            *
        from (
        select
           customer_user_id,
           install_time::date as install_date,
           install_time,
           country_code,
           media_source,
           type,
           row_number() over (partition by customer_user_id, install_date order by install_time) temp_interday_reinstall
        from
            reinstalls) as t
        where
            temp_interday_reinstall = 1
    ),

    last_active as (
        SELECT
            event_user,
            from_unixtime((last_active / 1000)::int) as last_active
        FROM
            impala_main_day.seg_players_{project_id}_pq
        WHERE
            hour = unix_timestamp('{d}'))

    select distinct
        '{p}' as app_short,
        customer_user_id as user_id,
        install_date,
        install_time,
        country_code,
        media_source,
        last_active
    from
        reinstalls_unique r
    join
        last_active l on r.customer_user_id = l.event_user
    """)
    cur.close()
    con.commit()
    con.close()

In [3]:
# projects = ('gs_as', 'gs_gp', 'hs_as', 'hs_gp', 'mm_as', 'mm_gp', 'fd_as', 'fd_gp')
projects = ('_gp',)
dates = pd.date_range('2021-01-01', '2023-09-30', freq='D')
all_slices = []
progress_bar = tqdm(total=len(all_slices))

for p in projects:
    for d in dates:
        all_slices.append((p, d))


pool_obj = multiprocessing.Pool(multiprocessing.cpu_count())
results = list(pool_obj.imap(insert_table, all_slices))
progress_bar.close()

0it [00:00, ?it/s]

NameError: name 'insert_table' is not defined

Process ForkPoolWorker-15:
Process ForkPoolWorker-6:
Process ForkPoolWorker-10:
Process ForkPoolWorker-9:
Process ForkPoolWorker-13:
Process ForkPoolWorker-16:
Process ForkPoolWorker-5:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-14:
Process ForkPoolWorker-1:
Process ForkPoolWorker-11:
Process ForkPoolWorker-8:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-12:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    s

In [4]:
# projects = ('gs_as', 'gs_gp', 'hs_as', 'hs_gp', 'mm_as', 'mm_gp', 'fd_as', 'fd_gp')
projects = ('ts_as', 'ts_gp')
dates = pd.date_range('2021-01-01', '2023-09-30', freq='D')
all_slices = []

for p in projects:
    for d in dates:
        all_slices.append((p, d))

progress_bar = tqdm(total=len(all_slices))

def execute_insert_table(slice):
    insert_table(slice)
    progress_bar.update(1)

# Установите количество потоков в пуле
max_workers = 20

# Используйте ThreadPoolExecutor для параллельного выполнения запросов
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    executor.map(execute_insert_table, all_slices)

progress_bar.close()

100%|█████████▉| 2004/2006 [23:45:05<01:25, 42.67s/it]     


In [5]:
secretId = 'jupyterhub/{}/credentials_serverless.json'.format(os.environ['JUPYTERHUB_USER'])
con = wr.redshift.connect(secret_id=secretId)
query = '''
select
    app_short,
    install_date,
    count(distinct app_short) i 
from
    ma_data.terentev_reinstalls_last_active
group by 
    1, 2
'''
df = wr.redshift.read_sql_query(query, con=con)
con.close()

pt = df.pivot_table(values='i', index='install_date', columns='app_short', aggfunc='sum')
null_values = pt.isna()
null_indices = null_values.stack().reset_index().values.tolist()
all_slices = [[x[1], x[0]] for x in null_indices if x[2] is True]
all_slices

[['hs_as', datetime.date(2021, 9, 1)],
 ['fd_as', datetime.date(2021, 9, 2)],
 ['gs_gp', datetime.date(2021, 9, 4)],
 ['gs_as', datetime.date(2021, 9, 5)],
 ['gs_gp', datetime.date(2021, 9, 6)],
 ['hs_as', datetime.date(2021, 9, 6)],
 ['hs_gp', datetime.date(2021, 9, 9)],
 ['mm_as', datetime.date(2021, 9, 12)],
 ['ts_gp', datetime.date(2021, 9, 14)],
 ['ts_gp', datetime.date(2021, 9, 21)],
 ['gs_gp', datetime.date(2021, 9, 29)],
 ['ts_as', datetime.date(2023, 1, 12)],
 ['ts_gp', datetime.date(2023, 3, 7)],
 ['fd_as', datetime.date(2023, 7, 31)],
 ['fd_gp', datetime.date(2023, 7, 31)],
 ['gs_as', datetime.date(2023, 7, 31)],
 ['gs_gp', datetime.date(2023, 7, 31)],
 ['hs_as', datetime.date(2023, 7, 31)],
 ['hs_gp', datetime.date(2023, 7, 31)],
 ['mm_as', datetime.date(2023, 7, 31)],
 ['mm_gp', datetime.date(2023, 7, 31)]]

In [6]:
progress_bar = tqdm(total=len(all_slices))

def execute_insert_table(slice):
    insert_table(slice)
    progress_bar.update(1)

# Установите количество потоков в пуле
max_workers = 16

# Используйте ThreadPoolExecutor для параллельного выполнения запросов
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    executor.map(execute_insert_table, all_slices)

progress_bar.close()

100%|██████████| 21/21 [12:10<00:00, 34.80s/it]
